
<h1 style="text-align:center;">Minerando Dados </h1>
<h2 style= "text-align:center;">Coletando e Trabalhando com Dados do Twitter (NLP)</h2>

* 1 - Criar conta no twitter - https://twitter.com/
* 2 - Applicar para conta de desenvoledor - https://developer.twitter.com/en/apply-for-access
* 3 - Criar o app

In [ ]:
from datetime import datetime
import json

In [1]:
consumer_key = "sua_consumer_key"
consumer_secret = "Seu consumer secret"

In [2]:
access_token = "your_token"
access_token_secret = "your_secret"

In [32]:
#! pip install TwitterSearch
# Referencias : 
# https://pypi.org/project/TwitterSearch/
# https://twittersearch.readthedocs.io/en/latest/advanced_usage_tso.html

In [48]:
from TwitterSearch import *

try:

    ts = TwitterSearch(  # Objeto TwitterSearch object
        consumer_key = consumer_key,
        consumer_secret = consumer_secret,
        access_token = access_token,
        access_token_secret = access_token_secret
     )

    tso = TwitterSearchOrder()   # cria objeto TwitterSearchOrder 
    tso.set_keywords(['ciencia de dados','machine learning'], or_operator = True)
    tso.set_language('pt')

    for tweet in ts.search_tweets_iterable(tso): # ts.search_tweets_iterable(tso) é um metadata
        
        print('created_at: ',tweet['created_at'], 'User_id: ',tweet['id_str'], 'Tweet: ',tweet['text'] ) 
        
        created_at = tweet['created_at']
        user_id = tweet['id_str']
        texto = tweet['text']
        
        with open("tweet.json", "a+") as output:
            
            data = { "created_at": created_at,
                     "User_id": user_id,
                     "tweet":texto }
            print(data)
            output.write("{}\n".format(json.dumps(data)))

except TwitterSearchException as e:
    print(e)

{'created_at': 'Thu Sep 10 15:57:11 +0000 2020', 'User_id': '1304086526540869632', 'tweet': 'RT @AlexisFonteyne: Antes de automatizar processos, fazer uso da inteligência artificial, aplicar machine learning, etc, temos que racional…'}
{'created_at': 'Thu Sep 10 15:55:08 +0000 2020', 'User_id': '1304086010830225408', 'tweet': 'RT @AlexisFonteyne: Antes de automatizar processos, fazer uso da inteligência artificial, aplicar machine learning, etc, temos que racional…'}
{'created_at': 'Thu Sep 10 15:54:37 +0000 2020', 'User_id': '1304085883281387528', 'tweet': 'Saúde id é uma plataforma que integra produtos e serviços de #saúde\n\n#produtosdesaúde\n#prevenção\n#app\n#tecnologia… https://t.co/B4q2tYRCnN'}
{'created_at': 'Thu Sep 10 15:47:59 +0000 2020', 'User_id': '1304084212308488192', 'tweet': 'RT @AlexisFonteyne: Antes de automatizar processos, fazer uso da inteligência artificial, aplicar machine learning, etc, temos que racional…'}
{'created_at': 'Thu Sep 10 15:47:59 +0000 2020', 'Use

In [1]:
import pandas as pd
df = pd.read_json('tweet.json',lines = True) 
df.head(10)

created_at              User_id  \
0 2020-09-10 15:23:03+00:00  1304077938623754240   
1 2020-09-10 15:19:55+00:00  1304077148207091712   
2 2020-09-10 14:53:47+00:00  1304070570036920320   
3 2020-09-10 14:48:55+00:00  1304069347174907904   
4 2020-09-10 14:41:02+00:00  1304067363348897792   
5 2020-09-10 14:38:15+00:00  1304066662019272704   
6 2020-09-10 13:46:54+00:00  1304053739272065024   
7 2020-09-10 13:41:24+00:00  1304052355818762240   
8 2020-09-10 13:40:03+00:00  1304052018043068416   
9 2020-09-10 13:33:03+00:00  1304050256494186496   

                                               tweet  
0  RT @canalpeixebabel: Imperdível!! Um evento de...  
1  RT @canalpeixebabel: Imperdível!! Um evento de...  
2  Na minha conversa com o prof. Joelson Sampaio ...  
3  Chamada ciência de dados para melhorar a saúde...  
4  RT @canalpeixebabel: Imperdível!! Um evento de...  
5  RT @canalpeixebabel: Imperdível!! Um evento de...  
6  RT @MartinsLuliano: Aproveitar que Excel está ...  
7  RT @canalpeixebabel: Imperdível!! Um evento de...  
8  RT @canalpeixebabel: Imperdível!! Um evento de...  
9  RT @canalpeixebabel: Imperdível!! Um evento de...

In [2]:
df.shape

(1308, 3)

Uma prática muito comum no Twitter é o retweet, ou seja, a republicação de tweets de outra pessoa. Dessa forma, provavelmente teremos muitos tweets repetidos na nossa base e por isso (nesse caso) devemos retira-los.

In [3]:
# Removendo os valores duplicados:
df.drop_duplicates(['tweet'], inplace=True)

In [4]:
df.shape

(515, 3)

#  NLTK (Natural Language Toolkit)

O NLTK (Natural Language Toolkit) é uma biblioteca que contém pacotes para fazer com que as máquinas entendam a linguagem humana. Abaixo vou listar alguns dos pacotes mais importantes.


In [5]:
import nltk

## Stopwords

- São palavras e termos frequentes porém que não possuem relevância nas sentenças.
    - Exemplos: as, os, um ,uma, com, de, da, para, etc …

O nltk possui uma lista de stopwords em 16 idiomas diferentes. Vamos criar função que remova todas as stopwords em Português.

In [6]:
# Função para remover Stopwords da nossa base:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [7]:
#stopwords = set(nltk.corpus.stopwords.words('portuguese'))
#stopwords
#instancia = 'Minerando Dados - o maior portal de Data Science do Brasil'
#palavras = [i for i in instancia.split() if not i in stopwords]
#palavras

## Stemming

É a técnica de reduzir uma palavra para o seu radical, mesmo que o radical em si não seja uma palavra válida no idioma. Removendo os sufixos e prefixos de uma palavra.

- Exemplo: o radical da palavra ilegal é ileg.

In [8]:
# Aplicando o stemming em nossa base:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

## Remove caracteres indesejados como links ,etc.

Vamos remover as pontuações e os links, pois eles não adiciona nenhuma informação extra ao tratar dados de texto.

In [10]:
import re

In [11]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

In [12]:
from bs4 import BeautifulSoup

def tweet_to_words(tweet):
    
    text = BeautifulSoup(tweet, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Zà-úÀ-Ú0-9]", " ", text.lower()) # Converte para minúsculo
    words = tweet_tokenizer.tokenize(text)
    
    return words

### Lemmatization

Reduz as palavras flexionadas adequadamente, garantindo que a palavra raiz pertença ao idioma. Determinando assim a palavra que representa seu lema.

Por exemplo execuções e executar, são formas da palavra execução, portanto, execução é o lema dessas palavras.

**Observação**: A função será mostrada a seguir, para fins de conhecimento, pois esse processo só está disponível para o idioma inglês, que não é o nosso caso.

In [13]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def Lemmatization(instancia):
  palavras = []
  for w in instancia.split():
    palavras.append(wordnet_lemmatizer.lemmatize(w))
  return (" ".join(palavras))

### Tokenização

É o processo de tokenizar ou dividir uma string ou textos em uma lista de tokens. O tokenizador padrão da nltk, usa o espaço para separara as palavras.

Exemplo: ‘Data Science do Zero’

Vai ficar: [‘Data’, ‘Science’, ‘do’, ‘Zero’]

No twitter é muito comum a utilização de @ e # além dos emoji 🙂 , se usarmos o tokenizador padrão na nossa base, ele vai dividir as string de forma incorreta, deixando de representar os tweets.

Veja um exemplo:

frase = ‘O video do @blogminerando é show! 🙂 🙂 😉 =D’

In [14]:
frase = "O video do @blogminerando é show! 🙂 🙂 😉 =D"

In [15]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer() 
tweet_tokenizer.tokenize(frase)

['O', 'video', 'do', '@blogminerando', 'é', 'show', '!', '🙂', '🙂', '😉', '=D']

## Tokenizar nosso Dataframe

In [16]:
df.head(3)

created_at              User_id  \
0 2020-09-10 15:23:03+00:00  1304077938623754240   
2 2020-09-10 14:53:47+00:00  1304070570036920320   
3 2020-09-10 14:48:55+00:00  1304069347174907904   

                                               tweet  
0  RT @canalpeixebabel: Imperdível!! Um evento de...  
2  Na minha conversa com o prof. Joelson Sampaio ...  
3  Chamada ciência de dados para melhorar a saúde...

In [17]:
tweets_tokenized = [tweet_tokenizer.tokenize(tweet) for tweet in df.tweet]

In [18]:
tweets_tokenized[17]

['📢',
 '📢',
 '📢',
 'ATENÇÃO',
 '!',
 '!',
 '!',
 '📢',
 '📢',
 '📢',
 'Conhece',
 'o',
 'novo',
 'curso',
 'da',
 'UTAD',
 ':',
 '✔',
 '️',
 'Licenciatura',
 'em',
 'Matemática',
 'Aplicada',
 'e',
 'Ciência',
 'de',
 'Dados',
 '📌',
 'Tens',
 '…',
 'https://t.co/9uID9dY8FP']

# Vamos usar somente 1 funções: remover os caracteres indesejados e stopwords

## Remover os caracteres indesejados e stopwords com nltk e re

In [19]:
def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [20]:
# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in df.tweet]

In [21]:
tweets[:10]

['rt @canalpeixebabel imperdível!! evento devas devs geral, dividido três trilhas arquitetura desenvolvimento net …',
 'conversa prof joelson sampaio falamos bastante sobre papel ciência dados formação eco…',
 'chamada ciência dados melhorar saúde maternoinfantil recebe propostas 28/09 saiba em…',
 'rt @martinsluliano aproveitar excel trends anunciar dou aulas particulares excel básico avançado além en…',
 'imperdível!! evento devas devs geral, dividido três trilhas arquitetura desenvolvimento net…',
 'rt @pachicodes olha massa galera arquitetura desenvolvimento net ciência dados, machine learning inteligência artificial…',
 'rt @liviagabos devbr women conference evento focado mulheres trans cis, aberto todos! 3 trilhas arquitetura…',
 'devbr women conference evento focado mulheres trans cis, aberto todos! 3 trilhas arqu…',
 'rt @comunicbr ciência dados vem orientando ações combate covid19 isso, aplicação #ia mostrou fundamental o…',
 'rt @giovannibassi dias 25 26/9 vai acontecer #devb

In [22]:
df['preprocessed'] = tweets
df.head()

created_at              User_id  \
0  2020-09-10 15:23:03+00:00  1304077938623754240   
2  2020-09-10 14:53:47+00:00  1304070570036920320   
3  2020-09-10 14:48:55+00:00  1304069347174907904   
6  2020-09-10 13:46:54+00:00  1304053739272065024   
12 2020-09-10 13:26:55+00:00  1304048709576130560   

                                                tweet  \
0   RT @canalpeixebabel: Imperdível!! Um evento de...   
2   Na minha conversa com o prof. Joelson Sampaio ...   
3   Chamada ciência de dados para melhorar a saúde...   
6   RT @MartinsLuliano: Aproveitar que Excel está ...   
12  Imperdível!! Um evento de devas para devs em g...   

                                         preprocessed  
0   rt @canalpeixebabel imperdível!! evento devas ...  
2   conversa prof joelson sampaio falamos bastante...  
3   chamada ciência dados melhorar saúde maternoin...  
6   rt @martinsluliano aproveitar excel trends anu...  
12  imperdível!! evento devas devs geral, dividido...

## remover caracteres indesejados com BeautifulSoup e re

In [23]:
from bs4 import BeautifulSoup

def prep_tweets(tweet):
    
    text = BeautifulSoup(tweet, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Zà-úÀ-Ú0-9]", " ", text.lower()) # Convert to lower case
    #words = tweet_tokenizer.tokenize(text)
    
    
    return text

In [28]:
df['cleaned_tweets'] =  [prep_tweets(tweet) for tweet in df.cleaned_tweets]
df.head()

created_at              User_id  \
0  2020-09-10 15:23:03+00:00  1304077938623754240   
2  2020-09-10 14:53:47+00:00  1304070570036920320   
3  2020-09-10 14:48:55+00:00  1304069347174907904   
6  2020-09-10 13:46:54+00:00  1304053739272065024   
12 2020-09-10 13:26:55+00:00  1304048709576130560   

                                                tweet  \
0   RT @canalpeixebabel: Imperdível!! Um evento de...   
2   Na minha conversa com o prof. Joelson Sampaio ...   
3   Chamada ciência de dados para melhorar a saúde...   
6   RT @MartinsLuliano: Aproveitar que Excel está ...   
12  Imperdível!! Um evento de devas para devs em g...   

                                         preprocessed  \
0   rt @canalpeixebabel imperdível!! evento devas ...   
2   conversa prof joelson sampaio falamos bastante...   
3   chamada ciência dados melhorar saúde maternoin...   
6   rt @martinsluliano aproveitar excel trends anu...   
12  imperdível!! evento devas devs geral, dividido...   

                                         clean_tweets  \
0   rt  canalpeixebabel imperdível   evento devas ...   
2   conversa prof joelson sampaio falamos bastante...   
3   chamada ciência dados melhorar saúde maternoin...   
6   rt  martinsluliano aproveitar excel trends anu...   
12  imperdível   evento devas devs geral  dividido...   

                                       cleaned_tweets  
0   rt  canalpeixebabel imperdível   evento devas ...  
2   conversa prof joelson sampaio falamos bastante...  
3   chamada ciência dados melhorar saúde maternoin...  
6   rt  martinsluliano aproveitar excel trends anu...  
12  imperdível   evento devas devs geral  dividido...

## Contar quantidade de palavras

In [29]:
# import CountVectorizer  para contar numero de vezes que uma palavra ocorre
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
# Create a matrix which shows the number of times specific words appear  on column text
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.cleaned_tweets)

In [52]:
#cv.get_feature_names()

In [54]:
#count_matrix.toarray()

In [53]:
# cria dataframe
word_count = pd.DataFrame(cv.get_feature_names(), columns = ['word'])
# Soma as palavras e converte em uma lista
word_count["count"] = count_matrix.sum(axis = 0).tolist()[0]
word_count = word_count.sort_values("count", ascending = False).reset_index(drop = True)
# 50 palavras mais usadas 
word_count[:50]

word  count
0        machine    212
1       learning    206
2          dados    171
3        ciência    148
4             rt    130
5            pra     50
6          sobre     47
7          curso     32
8         cursos     27
9          fazer     26
10    artificial     26
11  inteligência     25
12          data     24
13      aprender     22
14           vai     19
15    tecnologia     19
16     gratuitos     17
17          novo     17
18        python     17
19        fleury     16
20         saúde     15
21           ser     15
22            aí     15
23      trabalho     15
24   estatística     15
25       science     14
26      gratuito     14
27        online     14
28          hoje     13
29          quer     13
30          java     13
31           bom     13
32        artigo     13
33     algoritmo     13
34        evento     12
35            to     12
36      melhores     12
37         gente     12
38          tudo     12
39           ter     12
40         vamos     11
41            ia     11
42       mercado     11
43          área     11
44     português     11
45          deep     11
46           dia     11
47            10     11
48        usando     10
49   aprendizado     10